In [ ]:
from pathlib import Path
import os
import pandas as pd
from  shutil import copyfile
from tqdm import tqdm

### Run `detect.py` script for inference

In [ ]:
input_path = '../input/global-wheat-detection/test'
os.makedirs(input_path, exist_ok=True)

In [ ]:
# For testing large datasets
# for file in os.listdir('../input/global-wheat-detection/test'):
#     copyfile(f'../input/global-wheat-detection/test/{file}', f'./test_set/{file}')

# for file in os.listdir('../input/mnistasjpg/testSample/testSample'):
#     copyfile(f'../input/mnistasjpg/testSample/testSample/{file}', f'./test_set/{file}')

In [ ]:
!python ../input/yolov5ultralytics/yolov5/detect.py --source {input_path} --weights ../input/yolov5-wheat-weights/runs/train/wheat-detector2/weights/best.pt --imgsz 1024 --save-txt

### Denormalize inferences

In [ ]:
final_path = os.listdir('./runs/detect')[-1]
output_path = rf'runs/detect/{final_path}/labels'
output = pd.DataFrame(columns=['image_id', 'PredictionString'])

def denormalize(label):
    label = label[2:-1]
    label_lst = label.split(' ')
    label_float_lst = [int(float(i)*1024) for i in label_lst]
    xc, yc, w, h = label_float_lst
    label_float_lst = map(lambda x: str(int(x)), [xc - w/2, yc - h/2, w, h])
    denormalized_label_str = (' ').join(label_float_lst)
    label = '0.5 ' + denormalized_label_str
    return label

for file in os.listdir(input_path):
    image_id = Path(file).stem
    try:
        with open(output_path + f"/{image_id}.txt", "r") as file:
            content = file.readlines()
        denormalized_labels = [denormalize(line) for line in content]
        labels = (' ').join(denormalized_labels)
        output = output.append({'image_id': image_id, 'PredictionString': labels}, ignore_index=True)
    except FileNotFoundError:
        output = output.append({'image_id': image_id, 'PredictionString': ''}, ignore_index=True)


### Create output file

In [ ]:
print('made it here!!!')
print(output.head())

In [ ]:
!rm -r runs
output.to_csv('submission.csv', index=False)
print('finished!')

In [ ]:
'submission.csv' in os.listdir('.')

In [ ]:
# from matplotlib import pyplot as plt
# from matplotlib import patches
# import numpy as np
# from PIL import Image

In [ ]:
# with open('./runs/detect/exp/labels/51b3e36ab.txt', 'r') as file:
#     content = file.read()
# str_boxes = [box[2:] for box in content.split('\n')][:-1]
# int_boxes = [[int(float(x)*1024) for x in str_box.split(' ')] for str_box in str_boxes]
# int_boxes

# fig, ax = plt.subplots()
# for box in int_boxes:
#     xc, yc, w, h = box
#     x = xc - w/2
#     y = yc - h/2
#     ax.add_patch(patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'))
# ax.imshow(np.array(Image.open('../input/global-wheat-detection/test/51b3e36ab.jpg')))

In [ ]:

# for i, row in output.iterrows():
#     fig, ax = plt.subplots()
#     image_id = row.image_id
#     bbox = row.PredictionString
# #     with open(f'./runs/detect/exp/labels/{image_id}.txt', 'r') as file:
# #         content = file.read()
# #     str_boxes = [box[2:] for box in content.split('\n')][:-1]
# #     int_boxes = [[int(float(x)*1024) for x in str_box.split(' ')] for str_box in str_boxes]
# #     int_boxes
#     bboxes_int = [int(n) for i, n in enumerate(bbox.split(' ')) if i % 5 != 0] 
#     bboxes = [bboxes_int[i:i+4] for i in range(0, len(bboxes_int)-5, 4)]
#     for box in bboxes:
#         xc, yc, w, h = box;
#         x = xc - w/2
#         y = yc - h/2
#         ax.add_patch(patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none'))
#     ax.imshow(np.array(Image.open(f'{input_path}/{image_id}.jpg')))
#     plt.show()